In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import precision_score, recall_score, f1_score
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image

In [ ]:
!pip install keras tensorflow

In [ ]:
import os
import cv2
from skimage.feature import hog
from skimage import exposure
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, concatenate, Flatten
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image

In [ ]:
!pip install shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 9.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve
import joblib
import shap

In [ ]:
main_path = '/content/drive/My Drive/projects_data/fyp/Categorized_images'
dir_list = os.listdir(main_path)
print(dir_list)

['Bad', 'Neutral', 'Good', 'Very Bad', 'Very Good']


In [ ]:
labels = {'Very Bad':4, 'Bad':3, 'Neutral':2, 'Good':1, 'Very Good':0}

In [ ]:
image_list = {'Very Bad': [], 'Bad': [], 'Neutral': [], 'Good': [], 'Very Good': []}
for dir_name in dir_list:
    files_in_dir = os.listdir(f"{main_path}/{dir_name}")
    image_list[dir_name].extend(files_in_dir)

In [ ]:
for list_ in image_list:
  print(list_,len(image_list[list_]))

Very Bad 600
Bad 600
Neutral 600
Good 600
Very Good 600


In [ ]:
# Image Processing
def process_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (200, 200))
    return img
X = []
y = []

for label, img_paths in image_list.items():
    for img_name in img_paths:
        img_path = f"{main_path}/{label}/{img_name}"
        img = process_image(img_path)
        X.append(img)
        y.append(labels[label])

X = np.array(X)
y = np.array(y)


In [ ]:
# Normalizing the image data
X = X / 255.0

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Save the datasets
save_path = '/content/drive/MyDrive/projects_data/fyp/Labeled_Facebook_Posts/Saved_Datasets'
os.makedirs(save_path, exist_ok=True)
np.save(os.path.join(save_path, 'X_train1.npy'), X_train)
np.save(os.path.join(save_path, 'X_test1.npy'), X_test)
np.save(os.path.join(save_path, 'y_train1.npy'), y_train)
np.save(os.path.join(save_path, 'y_test1.npy'), y_test)

In [ ]:
# Model Definition
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Model Training
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=10,
                    batch_size=32)

Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 286s 4s/step - accuracy: 0.2358 - loss: 2.5305 - val_accuracy: 0.3350 - val_loss: 1.5106
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 312s 4s/step - accuracy: 0.3829 - loss: 1.4187 - val_accuracy: 0.3450 - val_loss: 1.4760
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 336s 4s/step - accuracy: 0.5005 - loss: 1.2504 - val_accuracy: 0.4267 - val_loss: 1.4315
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 318s 4s/step - accuracy: 0.6318 - loss: 0.9881 - val_accuracy: 0.3733 - val_loss: 1.5458
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 321s 4s/step - accuracy: 0.7768 - loss: 0.6683 - val_accuracy: 0.3783 - val_loss: 1.9465
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 280s 4s/step - accuracy: 0.8718 - loss: 0.4377 - val_accuracy: 0.3967 - val_loss: 2.3353
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 302s 3s/step - accuracy: 0.9388 - loss: 0.2228 - val_accuracy: 0.3833 - val_loss: 2.8895
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 280s 4s/step - accuracy: 0.9630 - loss: 0.1433 - val_accuracy: 0.3917 - v

In [ ]:
#Saving the model

from google.colab import drive
drive.mount('/content/drive')
save_path = '/content/drive/My Drive/models/Imageset.keras'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#load the model
from google.colab import drive
from keras.models import load_model
drive.mount('/content/drive')
load_path = '/content/drive/My Drive/models/Imageset.keras'
try:
    emotion_model = load_model(load_path)
    print("Model loaded successfully.")
    emotion_model.summary()
except Exception as e:
    print(f"Error loading model: {e}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Error loading model: File not found: filepath=/content/drive/My Drive/models/Imageset.keras. Please ensure the file is an accessible `.keras` zip file.


In [ ]:
# Model Evaluation
y_pred = np.argmax(model.predict(X_test), axis=1)

19/19 ━━━━━━━━━━━━━━━━━━━━ 16s 833ms/step


In [ ]:
# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Confusion Matrix:
[[40 34 11 23  5]
 [25 53  5 21  7]
 [25 17 37 32 11]
 [18 24  6 44 21]
 [27 30  3 40 41]]


In [ ]:
# Classification Report
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

Classification Report:
              precision    recall  f1-score   support

           0       0.30      0.35      0.32       113
           1       0.34      0.48      0.39       111
           2       0.60      0.30      0.40       122
           3       0.28      0.39      0.32       113
           4       0.48      0.29      0.36       141

    accuracy                           0.36       600
   macro avg       0.40      0.36      0.36       600
weighted avg       0.40      0.36      0.36       600

